## Borrower Reliability Research

The customer is the credit department of the bank. It is necessary to figure out whether the marital status and the number of the client's children affect the fact that the loan is repaid on time. Input data from the bank - statistics on clients' solvency.

The research results will be taken into  **credit scoring model** - a special system that assesses the ability of a potential borrower to repay a loan to a bank.

### 1. About data

In [1]:
#library import
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [2]:
#check data
data = pd.read_csv('/datasets/data.csv')
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

We have 21525 entries and 12 columns:

- children - the number of children in the family
- days_employed - work experience in days
- dob_years - client's age in years
- education - the level of education of the client
- education_id - educational level identifier
- family_status - marital status
- family_status_id - marital status identifier
- gender - customer gender
- income_type - type of employment
- debt - has debt or not
- total_income - monthly income
- purpose - the purpose of a loan


2 numeric float columns must be replaced to int. There are many negative values in days_employed, they should be positive. Also we see Nan or entries in different registers. Empties in days_employed could appear because the person did not work, and in total_income because he is not working now. Also it can be a human factor or old data.

### 2. Prepare data

### Fill values

In [4]:
#fill empty
data['education'] = data['education'].fillna(value='')
data['family_status'] = data['family_status'].fillna(value='')
data['gender'] = data['gender'].fillna(value='')
data['income_type'] = data['income_type'].fillna(value='')
data['purpose'] = data['purpose'].fillna(value='')

#fill Nan in total_income on income_type.median
median_income = data.groupby('income_type')['total_income'].agg(['median'])

def get_income_by_type(data , income_type):
    return (data.loc[income_type, 'median'])
    
def fill_NaN_income(row):
    result = row['total_income']
    if pd.isnull(result):
        result= get_income_by_type(median_income,row['income_type'])
    return result

data['total_income'] = data.apply(fill_NaN_income, axis=1)

#fill Nan in days_employed on income_type.median
median_days = data.groupby('income_type')['days_employed'].agg(['median'])

def get_days_by_type(data , income_type):
    return (data.loc[income_type, 'median'])
    
def fill_NaN_days(row):
    result = row['days_employed']
    if pd.isnull(result):
        result= get_income_by_type(median_days,row['income_type'])
    return result

data['days_employed'] = data.apply(fill_NaN_days, axis=1)

display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


## Conclusion

Nan in the object columns are empty now. The average value isn't fit.

In total income and days the median by income type.median is fill.

### Change data types

In [5]:
#float to int
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

#object to str.lower
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

#minus in days_employed
def plusday(row):
    if row < 0:
        return row * -1
    else:
        return row * 1      
data['days_employed'] = data['days_employed'].apply(plusday)

#minus in children
def pluskid(row):
    if row < 0:
        return row * -1
    else:
        return row * 1      
data['children'] = data['children'].apply(pluskid)
    
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


## Conclusion
Float converted to int because they are numeric values. 

In all object columns the case of letters is made small. 

Negative numbers us positive now.

### Duplicates

In [6]:
#data.duplicated().sum()
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Conclusion

We had 54 duplicates. It is small number, so they was deleted.

### Lemmatization

In [7]:
#column for lemm 
def lemma(row):
    return[m.lemmatize(row['purpose'])]
data['lemma'] = data.apply(lemma, axis=1)

#add entries in column for lemm
def lemma_for_table(row):
    
    purpos = row['lemma'][0]
    
    if purpos.count('жилье') > 0 or purpos.count('недвижимость') > 0:
            return 'жилье/недвижимость'
    if purpos.count('образование') > 0:
            return 'образование'
    if purpos.count('свадьба') > 0:
            return 'свадьба'
    return 'автомобиль'

data['lemmas'] = data.apply(lemma_for_table, axis=1)
display(data['lemmas'].value_counts())

жилье/недвижимость    10811
автомобиль             4306
образование            4013
свадьба                2324
Name: lemmas, dtype: int64

### Conclusion
- ***property 10811***
- ***car 4306***
- ***education 4013***
- ***wedding 2324***

### Categorization

In [8]:
#pivot table children/debt
data_pivot_children = data.pivot_table(index=['children'], columns=['debt'], values='family_status_id', aggfunc='count')
data_pivot_children['sum_kid'] = data_pivot_children.loc[:, '0':'1'].sum(axis=1)
display(data_pivot_children.fillna(0))

#procents for children
no_debts_procent_kid = (data_pivot_children[0] / data_pivot_children['sum_kid']) * 100
yes_debts_procent_kid = (data_pivot_children[1] / data_pivot_children['sum_kid'] ) * 100
display(no_debts_procent_kid.fillna(0))
display(yes_debts_procent_kid.fillna(0))

debt,0,1,sum_kid
children,,,
0,13028.0,1063.0,14091.0
1,4410.0,445.0,4855.0
2,1858.0,194.0,2052.0
3,303.0,27.0,330.0
4,37.0,4.0,41.0
5,9.0,0.0,9.0
20,68.0,8.0,76.0


children
0      92.456178
1      90.834192
2      90.545809
3      91.818182
4      90.243902
5     100.000000
20     89.473684
dtype: float64

children
0      7.543822
1      9.165808
2      9.454191
3      8.181818
4      9.756098
5      0.000000
20    10.526316
dtype: float64

In [9]:
#pivot table family_status/debt
data_pivot_family_status = data.pivot_table(index=['family_status'], columns=['debt'], values='family_status_id', aggfunc='count')
data_pivot_family_status['sum_fam'] = data_pivot_family_status.loc[:, '0':'1'].sum(axis=1)
display(data_pivot_family_status.fillna(0))

#procents for family_status
no_debts_procent_fam = (data_pivot_family_status[0] / data_pivot_family_status['sum_fam']) * 100
yes_debts_procent_fam = (data_pivot_family_status[1] / data_pivot_family_status['sum_fam']) * 100
display(no_debts_procent_fam.fillna(0))
display(yes_debts_procent_fam.fillna(0))

debt,0,1,sum_fam
family_status,,,
в разводе,1110,85,1195
вдовец / вдова,896,63,959
гражданский брак,3763,388,4151
женат / замужем,11408,931,12339
не женат / не замужем,2536,274,2810


family_status
в разводе                92.887029
вдовец / вдова           93.430657
гражданский брак         90.652855
женат / замужем          92.454818
не женат / не замужем    90.249110
dtype: float64

family_status
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
не женат / не замужем    9.750890
dtype: float64

- ***в разводе - divorced***
- ***вдовец / вдова - widower / widow***
- ***гражданский брак - partner***
- ***женат / замужем - married***
- ***не женат / не замужем - single***

In [10]:
#function for total_income
def debts(row):
    
    income = row['total_income']
    
    if income < 50000:
            return 'low'
    if income > 50001 and income < 100000:
            return 'medium'
    if income > 100001 and income < 170000:
            return 'upper'
    return 'high'

data['debts'] = data.apply(debts, axis=1)

#pivot table total_income/debt
data_pivot_total_income = data.pivot_table(index=['debts'], columns=['debt'], values='family_status_id', aggfunc='count')
data_pivot_total_income['sum_incom'] = data_pivot_total_income.loc[:, '0':'1'].sum(axis=1)
display(data_pivot_total_income.fillna(0))

#procents for total_income
no_debts_procent_income = (data_pivot_total_income[0] / data_pivot_total_income['sum_incom']) * 100
yes_debts_procent_income = (data_pivot_total_income[1] / data_pivot_total_income['sum_incom']) * 100
display(no_debts_procent_income.fillna(0))
display(yes_debts_procent_income.fillna(0))

debt,0,1,sum_incom
debts,,,
high,7141,577,7718
low,349,23,372
medium,3760,331,4091
upper,8463,810,9273


debts
high      92.523970
low       93.817204
medium    91.909069
upper     91.264963
dtype: float64

debts
high      7.476030
low       6.182796
medium    8.090931
upper     8.735037
dtype: float64

In [11]:
#pivot table purpose/debt
data_pivot_lemmas = data.pivot_table(index=['lemmas'], columns=['debt'], values='family_status_id', aggfunc='count')
data_pivot_lemmas['sum_lemma'] = data_pivot_lemmas.loc[:, '0':'1'].sum(axis=1)
display(data_pivot_lemmas.fillna(0))

#procents for purpose
no_debts_procent_lemma = (data_pivot_lemmas[0] / data_pivot_lemmas['sum_lemma']) * 100
yes_debts_procent_lemma = (data_pivot_lemmas[1] / data_pivot_lemmas['sum_lemma']) * 100
display(no_debts_procent_lemma.fillna(0))
display(yes_debts_procent_lemma.fillna(0))

debt,0,1,sum_lemma
lemmas,,,
автомобиль,3903,403,4306
жилье/недвижимость,10029,782,10811
образование,3643,370,4013
свадьба,2138,186,2324


lemmas
автомобиль            90.640966
жилье/недвижимость    92.766627
образование           90.779965
свадьба               91.996558
dtype: float64

lemmas
автомобиль            9.359034
жилье/недвижимость    7.233373
образование           9.220035
свадьба               8.003442
dtype: float64

- ***автомобиль - car***
- ***жилье/недвижимость - property***
- ***образование - education***
- ***свадьба - wedding***     

### Conclusion

Borrowers are divided into several categories according to the requests below. 

The number of children in the family has been calculated, so the debt sample is more accurate. 

Marital status is also calculated. 

The income is divided into 4 approximate parts, and the credit purpose are based on the column with lemmas from the previous task.

### 3. Answer the questions

- ***Is there a dependence between the children and the repayment of the loan on time?***

The majority of people with or without children do not have debts:
- 100% with 5 children have no debt 
- 92% without children 
- 91% with three children.

Most of the borrowers with debts have 1, 2 and 4 children: all about 9%.

Families with 5 children most likely have great support from relatives and the state, so they manage to repay debts on time. 

People without children are in the top because most likely they are better at calculating their budget for loan and have fewer force majeure situations than people with 1,2 or 4 children.

- ***Is there a dependence between the family status and the repayment of the loan on time?***

The majority (93% and 92%) of people who divorced and widowers do not have credit debts. 

In third place are married people (92%). 

Most of the borrowers with debts are peoople who live in a partnership and single (9% each).

Divorced people or widowers usually have experience and know how to calculate their budget. 

Married people are reliable borrowers, in the event of force majeure husband/wife can help.

- ***Is there a dependence between the income and the repayment of the loan on time?***

The distribution is softer: people of all income categories generally have no debt (more than 91%).

The largest percentage of those who have it: people with middle and upper middle income (8%). It seems to me that there is no clear dependence here.

- ***How do different purposes of a loan affect its return on time?***

It's look like a table with income. 

Regardless of the goals, people are mostly free of debt (more than 90%), but the largest percentage of those with loans for cars and education.

Most likely because the car requires constant investments, in contrast to property. 

There are also a lot of debts from those taking a loan for education, perhaps there is not enough salary to pay off while a person improves his qualifications.

- ***Conclusion***

In general, people are more likely to pay back their debts on time.

The most responsible people are those who are married (divorced and widowed) without children or with five children with high or low income who are planning property transactions.